In [15]:
import pandas as pd
import cv2 as cv
import os

In [16]:
with open(r'data\bressay\sets\test.txt', 'r') as file:
    test_content = file.readlines()

with open(r'data\bressay\sets\training.txt', 'r') as file:
    training_content = file.readlines()

with open(r'data\bressay\sets\validation.txt', 'r') as file:
    validation_content = file.readlines()



In [ ]:
# counter_rasura = 0
# if "--xxx---" in data_text:
#     counter_rasura += 1

# Extracting line dataset

In [26]:
lines_directory = r'data\bressay\data\lines'
test_files = {}
training_files = {}
validation_files = {}

def collect_file_paths(dir_name, base_path):
    file_paths = {'txt': []}
    full_path = os.path.join(base_path, dir_name)
    if os.path.isdir(full_path):
        for file in os.listdir(full_path):
            if file.endswith('.txt'):
                file_paths['txt'].append(os.path.join(full_path, file))
    return file_paths



for dir_name in test_content:
    dir_name = dir_name.strip()
    test_files[dir_name] = collect_file_paths(dir_name, lines_directory)

for dir_name in training_content:
    dir_name = dir_name.strip()
    training_files[dir_name] = collect_file_paths(dir_name, lines_directory)

for dir_name in validation_content:
    dir_name = dir_name.strip()
    validation_files[dir_name] = collect_file_paths(dir_name, lines_directory)


    

In [27]:
list(training_files.items())[:2]

[('0087',
  {'txt': ['data\\bressay\\data\\lines\\0087\\0087_01_01.txt',
    'data\\bressay\\data\\lines\\0087\\0087_01_02.txt',
    'data\\bressay\\data\\lines\\0087\\0087_01_03.txt',
    'data\\bressay\\data\\lines\\0087\\0087_01_04.txt',
    'data\\bressay\\data\\lines\\0087\\0087_01_05.txt',
    'data\\bressay\\data\\lines\\0087\\0087_01_06.txt',
    'data\\bressay\\data\\lines\\0087\\0087_02_01.txt',
    'data\\bressay\\data\\lines\\0087\\0087_02_02.txt',
    'data\\bressay\\data\\lines\\0087\\0087_02_03.txt',
    'data\\bressay\\data\\lines\\0087\\0087_02_04.txt',
    'data\\bressay\\data\\lines\\0087\\0087_02_05.txt',
    'data\\bressay\\data\\lines\\0087\\0087_02_06.txt',
    'data\\bressay\\data\\lines\\0087\\0087_02_07.txt',
    'data\\bressay\\data\\lines\\0087\\0087_02_08.txt',
    'data\\bressay\\data\\lines\\0087\\0087_03_01.txt',
    'data\\bressay\\data\\lines\\0087\\0087_03_02.txt',
    'data\\bressay\\data\\lines\\0087\\0087_03_03.txt',
    'data\\bressay\\data\\lines